### K-means 클러스터링을 시도하여 SELLER 군집화 시도

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [ ]:
# 샘플 데이터프레임 생성

df = pd.read_csv('./data_in/total_df.csv')


In [ ]:
# 클러스터링에 사용할 특징 선택

features = ['Order_item_id', 'Price',
       'Freight_value',
       'Payment_sequential', 'Payment_installments',
       'Payment_value', 'Product_weight_g',
       'Product_length_cm', 'Product_height_cm', 'Product_width_cm',
       'Seller_zipcode_prefix',
       'Customer_zipcode_prefix',
       'Review_score',
       'Items_per_order', 'Revenue']

In [ ]:
# 원하는 컬럼만을 선택

column_filter = ['Order_item_id', 'Price',
       'Freight_value',
       'Payment_sequential', 'Payment_installments',
       'Payment_value', 'Product_weight_g',
       'Product_length_cm', 'Product_height_cm', 'Product_width_cm',
       'Seller_zipcode_prefix',
       'Customer_zipcode_prefix',
       'Review_score',
       'Items_per_order', 'Revenue']

df_plus = df[column_filter]

In [ ]:
# 결측치 제거

df_plus['Review_score'].fillna(0, inplace= True)

# 데이터 클리닝: 'Unknown' 값을 NaN으로 변환 후 제거

df_plus = df_plus.replace('Unknown', np.nan).dropna()

In [ ]:
# 특징 표준화
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_plus[features])

# k-means 클러스터링
n_clusters = 2  # 클러스터 개수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=3)
df_plus['cluster'] = kmeans.fit_predict(df_scaled)

In [ ]:
import seaborn as sns

# 상관 관계 행렬 계산
correlation_matrix = df_plus.corr()

# 상관 관계 히트맵 시각화
sns.heatmap(correlation_matrix, annot=True, annot_kws={'size' : 6})
plt.figure(figsize= (50,50))
plt.show()


In [ ]:
# 클러스터 시각화
for cluster in range(n_clusters):
    plt.scatter(df_plus[df_plus['cluster'] == cluster]['Revenue'],
                df_plus[df_plus['cluster'] == cluster]['Order_item_id'], # 상관계수가 낮고 분산이 높은 컬럼 선택
                label=f'Cluster {cluster + 1}')

plt.xlabel('Revenue')
plt.ylabel('Order_item_id')
plt.title('K-Means Clustering')
plt.legend()
plt.show()

# 클러스터 중심 출력
print("Cluster centers:")
print(kmeans.cluster_centers_)


In [ ]:
correlation_matrix